#### 이제 은닉층을 쌓아보자!

- 모델 파라미터 개수 구하기

    입력층 유닛이 5개, 은닉층 유닛을 10개로 지정한다고 가정해보자.

    - 입력층 -> 은닉층으로 갈 때 만들어지는 모델 파라미터의 개수
        - 가중치(weights) = 5 * 10 = 50
        - 편향(bias) = 10
        - 총합: 50 + 10 = 60개

    출력층 유닛을 3개로 지정한다고 가정하면:
    - 은닉층 -> 출력층으로 갈 때 만들어지는 모델 파라미터의 개수
        - 가중치(weights) = 10 * 3 = 30
        - 편향(bias) = 3
        - 총합: 30 + 3 = 33개

    총 생성되는 모델 파라미터 개수 = 60 + 33 = 93개

**OVR (One-vs-Rest)**
> 은닉층은 입력 특징을 비선형으로 변형하여 더 복잡한 패턴을 학습할 수 있게 한다
- 이진 분류 활성화 함수 사용
- ex) 분류할 클래스가 3개라고 가정 (A, B, C)

    OVR 방식은 다음과 같은 3개의 이진 분류기 생성
    1. A vs 나머지(B, C)
    2. B vs 나머지(A, C)
    3. C vs 나머지(A, B)
    
    각 모델은 자기가 맡은 클래스인지 아닌지만 판단 (1 or rest)

In [4]:
import keras

# 데이터셋 불러와서, 훈련용/테스트용 세트 분리
(train_input, train_target), (test_input, test_target) =\
    keras.datasets.fashion_mnist.load_data()

In [5]:
# 정규화 (0~1)
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28 * 28)

In [6]:
# 훈련용/검증용 세트 분리
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

In [ ]:
# 입력층, 밀집층(은닉층), 출력층 구성

inputs = keras.layers.Input(shape=(28*28,))  # 입력층: 28x28 이미지를 1D 벡터(784차원)로 처리

dense1 = keras.layers.Dense(100, activation='sigmoid')  # 은닉층1: 유닛 100개
# 활성화 함수 - sigmoid(이진 분류)로 설정한 이유:
# 은닉층은 입력 특징을 비선형으로 변형하여 더 복잡한 패턴을 학습할 수 있게 한다

outputs = keras.layers.Dense(10, activation='softmax')  # 출력층: 클래스 10개, 확률 출력

model = keras.Sequential([inputs, dense1, outputs])  # 순차 모델로 층 연결

model.summary()  # 모델 구조 요약

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

- 입력층 -> 은닉층\
784 * 100 + 100 = 78,500개

- 은닉층 -> 출력층\
100 * 10 + 10 = 1,010개

- 총 파라미터 개수 = 78,500 + 1,010 = 79,510개

In [9]:
# 컴파일 - 손실 함수(loss), 옵티마이저(optimizer) 등 설정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7538 - loss: 0.7657  
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.8478 - loss: 0.4236
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 991us/step - accuracy: 0.8626 - loss: 0.3810
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 969us/step - accuracy: 0.8705 - loss: 0.3546
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 998us/step - accuracy: 0.8784 - loss: 0.3333


은닉층을 하나 더 쌓아보자!

In [11]:
# 입력층, 밀집층(은닉층), 출력층 구성

inputs = keras.layers.Input(shape=(28*28,))  # 입력층: 28x28 이미지를 1D 벡터(784차원)로 처리

dense1 = keras.layers.Dense(100, activation='sigmoid')  # 은닉층1: 유닛 100개
dense2 = keras.layers.Dense(50, activation='sigmoid')   # 은닉층2: 유닛 50개
# 활성화 함수 - sigmoid(이진 분류)로 설정한 이유:
# 은닉층은 입력 특징을 비선형으로 변형하여 더 복잡한 패턴을 학습할 수 있게 한다

outputs = keras.layers.Dense(10, activation='softmax')  # 출력층: 클래스 10개, 확률 출력

model = keras.Sequential([inputs, dense1, dense2, outputs])  # 순차 모델로 층 연결

model.summary()  # 모델 구조 요약

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 100)            │        78,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,060 (328.36 KB)

 Trainable params: 84,060 (328.36 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# 컴파일 - 손실 함수(loss), 옵티마이저(optimizer) 등 설정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6804 - loss: 1.0278  
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8446 - loss: 0.4343
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8590 - loss: 0.3894
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8702 - loss: 0.3627
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8773 - loss: 0.3417


#### ReLU(Rectified Linear Unit) 함수
$$
\mathrm{ReLU}(x) = \max(0, x)
$$

즉,
- 𝑥 > 0 이면 그대로 출력
- 𝑥 ≤ 0 이면 0 출력

In [14]:
# 입력층, 밀집층(은닉층), 출력층 구성

inputs = keras.layers.Input(shape=(28*28,))  # 입력층: 28x28 이미지를 1D 벡터(784차원)로 처리

dense1 = keras.layers.Dense(100, activation='relu')  # 은닉층1: 유닛 100개
dense2 = keras.layers.Dense(50, activation='relu')   # 은닉층2: 유닛 50개

outputs = keras.layers.Dense(10, activation='softmax')  # 출력층: 클래스 10개, 확률 출력

model = keras.Sequential([inputs, dense1, dense2, outputs])  # 순차 모델로 층 연결

# 컴파일 - 손실 함수(loss), 옵티마이저(optimizer) 등 설정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7612 - loss: 0.6819
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8561 - loss: 0.3975  
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8698 - loss: 0.3596
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8782 - loss: 0.3346
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8850 - loss: 0.3216


좌->우 방향으로 쭉 나열하는 것보다, 상->하 방향이 더 보기가 좋다.\
아래와 같이 model에 .add하는 방식을 사용하자. (성능 차이는 X)

In [15]:
# 입력층, 밀집층(은닉층), 출력층 구성

model = keras.Sequential()
model.add(keras.layers.Input(shape=(28*28,)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 컴파일 - 손실 함수(loss), 옵티마이저(optimizer) 등 설정
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 학습
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7645 - loss: 0.6757
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 995us/step - accuracy: 0.8539 - loss: 0.3947
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8703 - loss: 0.3579
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8766 - loss: 0.3384
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8819 - loss: 0.3298


In [16]:
# 모델 최종 평가
test_scaled = test_input / 255.0
test_scaled = test_scaled.reshape(-1, 28 * 28)
model.evaluate(test_scaled, test_target)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - accuracy: 0.8746 - loss: 0.3743


[0.37203270196914673, 0.8765000104904175]

In [19]:
import numpy as np

# 예측하기
predictions = model.predict(test_scaled[:10])
print(np.round(predictions, decimals=3))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[[0.    0.    0.    0.    0.    0.007 0.    0.222 0.    0.771]
 [0.    0.    0.991 0.    0.003 0.    0.006 0.    0.    0.   ]
 [0.    1.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    1.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.137 0.    0.045 0.004 0.004 0.    0.81  0.    0.001 0.   ]
 [0.    1.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.015 0.    0.985 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.003 0.    0.997 0.    0.   ]]


```text
[[0.    0.    0.    0.    0.    0.007 0.    0.222 0.    0.771]  -> 0.771 (9번째 인덱스)
 [0.    0.    0.991 0.    0.003 0.    0.006 0.    0.    0.   ]  -> 0.991 (2번째 인덱스)
 [0.    1.    0.    0.    0.    0.    0.    0.    0.    0.   ]  -> 1 (1번째 인덱스)
 [0.    1.    0.    0.    0.    0.    0.    0.    0.    0.   ]  -> 
 [0.137 0.    0.045 0.004 0.004 0.    0.81  0.    0.001 0.   ]
 [0.    1.    0.    0.    0.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.015 0.    0.985 0.    0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.003 0.    0.997 0.    0.   ]]
```

각 배열에서 가장 큰 확률이 판단 결과이다.\
판단 결과를 배열로 만드려면?\
-> argmax() 사용 (arg: index를 가져온다)\
-> 각 열 중에서 가장 큰 값을 가져온다 => `axis=1`

In [20]:
np.argmax(predictions, axis=1)

array([9, 2, 1, 1, 6, 1, 4, 6, 5, 7])

In [ ]:
# Flatten() 층, 다차원 넘파이 배열 -> 1차원적인 입력으로 변환

train_scaled = train_input / 255.0
train_scaled.shape

In [25]:
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

ValueError: Found input variables with inconsistent numbers of samples: [60000, 48000]

뭔가 꼬인 듯하여... ex04로 넘어가서 `Flatten()`을 써 보자!